In [1]:
# 读写文件
''' 
到⽬前为⽌，我们讨论了如何处理数据，以及如何构建、训练和测试深度学习模型。然⽽，有时我们希望保
存训练的模型，以备将来在各种环境中使⽤（⽐如在部署中进⾏预测）。此外，当运⾏⼀个耗时较⻓的训练过
程时，最佳的做法是定期保存中间结果，以确保在服务器电源被不⼩
'''
# 加载和保存张量
''' 
对于单个张量，我们可以直接调⽤load和save函数分别读写它们。这两个函数都要求我们提供⼀个名称，
save要求将要保存的变量作为输⼊。
'''
import torch
from torch import nn
from torch.nn import functional as F
x = torch.arange(4) # 返回大小的一维张量,默认步长为1
torch.save(x, 'x-file') # 默认存储到当前文件目录之中

In [2]:
# 将存储在文件中的数据读回内存
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [3]:
# 存储一个张量列表,然后把他们读回内存
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [4]:
# 我们甚⾄可以写⼊或读取从字符串映射到张量的字典
# 当我们要读取或写⼊模型中的所有权重时，这很⽅便
mydict = {'x': x, 'y': y} # 从字符串映射到张量的字典
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [5]:
# 加载和保存模型参数
''' 
保存单个权重向量（或其他张量）确实有⽤，但是如果我们想保存整个模型，并在以后加载它们，单独保存
每个向量则会变得很⿇烦。毕竟，我们可能有数百个参数散布在各处。因此，深度学习框架提供了内置函数
来保存和加载整个⽹络。需要注意的⼀个重要细节是，这将保存模型的参数⽽不是保存整个模型。例如，如
果我们有⼀个3层多层感知机，我们需要单独指定架构。因为模型本⾝可以包含任意代码，所以模型本⾝难以
序列化。因此，为了恢复模型，我们需要⽤代码⽣成架构，然后从磁盘加载参数。让我们从熟悉的多层感知
机开始尝试⼀下。
'''
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
net = MLP()
X = torch.randn(size=(2, 20)) # 正态分布 输入
Y = net(X)


In [6]:
# 接下来，我们将模型的参数存储在⼀个叫做“mlp.params”的⽂件中
torch.save(net.state_dict(), 'mlp.params')
net.state_dict() # 网络参数字典

OrderedDict([('hidden.weight',
              tensor([[ 0.1829,  0.0009, -0.1113,  ..., -0.0701,  0.1487, -0.1286],
                      [-0.0509,  0.0267,  0.0162,  ..., -0.1907,  0.2216, -0.0468],
                      [ 0.1317,  0.1097, -0.1184,  ...,  0.0400, -0.0939,  0.1119],
                      ...,
                      [-0.0271, -0.1276, -0.0860,  ...,  0.2049,  0.2191, -0.1566],
                      [-0.2069, -0.0786, -0.1639,  ...,  0.1856,  0.1138,  0.0470],
                      [ 0.1154, -0.0364,  0.1285,  ..., -0.1350, -0.1268, -0.0480]])),
             ('hidden.bias',
              tensor([-0.1493,  0.0345, -0.1249, -0.1243,  0.0973,  0.0539, -0.1679, -0.2109,
                      -0.2101,  0.0525,  0.0609, -0.1675,  0.1996, -0.0947,  0.0683, -0.0179,
                       0.0647, -0.0620, -0.1576, -0.1882,  0.1437, -0.1700,  0.0815,  0.1956,
                       0.2103, -0.1531, -0.1655,  0.0288,  0.1789, -0.1028, -0.1983,  0.0498,
                      -0.1873,

In [8]:
''' 
为了恢复模型，我们实例化了原始多层感知机模型的⼀个备份。这⾥我们不需要随机初始化模型参数，⽽是
直接读取⽂件中存储的参数

train()  启用 BatchNormalization 和 Dropout
eval() 不启用 BatchNormalization 和 Dropout
'''
# 网络层中自动执行参数初始化 
# self.weight = nn.Parameter(torch.randn(in_units, units))
'''
框架的延后初始化（defers initialization），
即直到数据第⼀次通过模型传递时，
框架才会动态地推断出每个层的⼤⼩
'''
clone = MLP() # 克隆MLP架构
clone.load_state_dict(torch.load('mlp.params')) # 加载参数 延后初始化
clone.eval() # model.eval() 等同于 self.train(False) 即评估模式,而非训练模式

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [16]:
# 由于两个实例具有相同的模型参数，在输⼊相同的X时，两个实例的计算结果应该相同
# 让我们来验证⼀下
Y_clone = clone(X) # clone = MLP()  net = MLP()
Y_clone == Y       # Y = net(X)

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [ ]:
''' 
⼩结
• save和load函数可⽤于张量对象的⽂件读写。
• 我们可以通过参数字典保存和加载⽹络的全部参数。
• 保存架构必须在代码中完成，⽽不是在参数中完成。
'''